This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

---

In [1]:
%cd /code

/code


---

In [9]:
from dotenv import load_dotenv
load_dotenv()

import textwrap

def mprint(string: str, max_width: int = 80) -> None:
    """Print `string` with a maximum widgth."""
    wrapped_string = textwrap.fill(string, max_width)
    print(wrapped_string)

# OpenAI Chat

Simple example using `GPT-3.5`:

In [14]:
from llm_chain.models import OpenAIChat

model = OpenAIChat(model_name='gpt-3.5-turbo')
response = model("What is the meaning of life?")
mprint(response)

The meaning of life is a philosophical question that has been debated by
scholars, theologians, and philosophers for centuries. There is no one
definitive answer to this question, as it can vary depending on one's beliefs,
values, and experiences. Some people believe that the meaning of life is to seek
happiness, while others believe it is to fulfill a specific purpose or destiny.
Ultimately, the meaning of life is a personal and subjective concept that each
individual must determine for themselves.


---

## Streaming

We can stream the response by using a callback which takes a single parameter of type `StreamingRecord` which has a `response` property containing each token streamed. In the example below, we print simply print the response and end the printed result with the `|` character.

In [12]:
from llm_chain.models import OpenAIChat

model = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|')
)
response = model("What is the meaning of life?")

The| meaning| of| life| is| a| philosophical| question| that| has| been| debated| by| scholars|,| theolog|ians|,| and| philosophers| for| centuries|.| There| is| no| one| definitive| answer| to| this| question|,| as| it| can| vary| depending| on| one|'s| beliefs|,| values|,| and| experiences|.| Some| people| believe| that| the| meaning| of| life| is| to| seek| happiness|,| while| others| believe| it| is| to| fulfill| a| specific| purpose| or| destiny|.| Ultimately|,| the| meaning| of| life| is| a| personal| and| subjective| concept| that| each| individual| must| determine| for| themselves|.|

Like the previous example, the full text is returned at the end:

In [13]:
mprint(response)

The meaning of life is a philosophical question that has been debated by
scholars, theologians, and philosophers for centuries. There is no one
definitive answer to this question, as it can vary depending on one's beliefs,
values, and experiences. Some people believe that the meaning of life is to seek
happiness, while others believe it is to fulfill a specific purpose or destiny.
Ultimately, the meaning of life is a personal and subjective concept that each
individual must determine for themselves.


---

## Usage & Cost

model.total_cost

In [28]:
from llm_chain.models import OpenAIChat

model = OpenAIChat(model_name='gpt-3.5-turbo')
model("What is the capital of France?")

'The capital of France is Paris.'

In [29]:
print(f"Total Cost:            ${model.cost:.5f}")
print(f"Total Tokens:          {model.total_tokens:,}")
print(f"Total Prompt Tokens:   {model.prompt_tokens:,}")
print(f"Total Response Tokens: {model.response_tokens:,}")

Total Cost:            $0.00005
Total Tokens:          33
Total Prompt Tokens:   26
Total Response Tokens: 7


If we use the same model/object again, the cost/usage will be incremented accordingly.

In [30]:
model("What is the capital of Germany?")

'The capital of Germany is Berlin.'

In [32]:
print(f"Total Cost:            ${model.cost:.5f}")
print(f"Total Tokens:          {model.total_tokens:,}")
print(f"Total Prompt Tokens:   {model.prompt_tokens:,}")
print(f"Total Response Tokens: {model.response_tokens:,}")

Total Cost:            $0.00014
Total Tokens:          90
Total Prompt Tokens:   76
Total Response Tokens: 14


## History

We can use the `history` property to get the prompt/response and cost/usage for each of the messages used by the model/object. 

There are two `MessageRecord` items in the list. The first item corresponds to the first question and the second item corresponds to the second question.

In [33]:
model.history

[MessageRecord(uuid='3d2d5acc-4da7-4385-b941-a3840b5fc929', timestamp='2023-06-23 04:55:22.213', metadata={'model_name': 'gpt-3.5-turbo'}, total_tokens=33, cost=5.3e-05, prompt='What is the capital of France?', response='The capital of France is Paris.', prompt_tokens=26, response_tokens=7),
 MessageRecord(uuid='fa7e5d93-a2f7-4498-a946-a1f57c68624a', timestamp='2023-06-23 04:55:25.517', metadata={'model_name': 'gpt-3.5-turbo'}, total_tokens=57, cost=8.900000000000001e-05, prompt='What is the capital of Germany?', response='The capital of Germany is Berlin.', prompt_tokens=50, response_tokens=7)]

In [39]:
print(f"prompt: {model.history[0].prompt}")
print(f"prompt: {model.history[0].response_tokens}")
print(f"Total Cost:            ${model.history[0].cost:.5f}")
print(f"Total Tokens:          {model.history[0].total_tokens:,}")
print(f"Total Prompt Tokens:   {model.history[0].prompt_tokens:,}")
print(f"Total Response Tokens: {model.history[0].response_tokens:,}")

prompt: What is the capital of France?
prompt: 7
Total Cost:            $0.00005
Total Tokens:          33
Total Prompt Tokens:   26
Total Response Tokens: 7


In [40]:
print(f"prompt: {model.history[1].prompt}")
print(f"prompt: {model.history[1].response_tokens}")
print(f"Total Cost:            ${model.history[1].cost:.5f}")
print(f"Total Tokens:          {model.history[1].total_tokens:,}")
print(f"Total Prompt Tokens:   {model.history[1].prompt_tokens:,}")
print(f"Total Response Tokens: {model.history[1].response_tokens:,}")

prompt: What is the capital of Germany?
prompt: 7
Total Cost:            $0.00009
Total Tokens:          57
Total Prompt Tokens:   50
Total Response Tokens: 7
